# ORM - sqlalchemy

In [ ]:
! pip show sqlalchemy

In [1]:
import sqlalchemy

In [2]:
sqlalchemy.__version__

'1.4.38'

#### 구조: DB API -> 코어 -> ORM

#### 코어
- 항상 엔진을 만들면서 시작한다.
- engine = connection + dialect
- craete_engine echo 옵션 == 로그 보이게하기
- lazy connecting, 실제 엔진에 접속할 때 연결한다. 파일이 바로 생기지 않는다.

In [4]:
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import Table, Column, ForeignKey
from sqlalchemy.types import Integer, Text
from sqlalchemy.sql import select, insert, update

In [9]:
!ls *.db
!diri *.db

playlist.db sns.db
zsh:1: command not found: diri


In [5]:
# 아직 db 안 만들어진것 확인하기
engine = create_engine('sqlite:///orm.db', echo=True)

1               2           3             4
Python      |   ORM         RDBMS         DB
------      |   Core       SQLite        FILE
       Obj  |                                  
(class => MetaData)

메타데이터에 클래스가 어떻게 등록되는지만 알면된다.
메타데이터를 갖고 있는것 1
일을 하는 것 2 

In [6]:
from sqlalchemy.schema import MetaData
meta = MetaData()

In [7]:
meta.tables

FacadeDict({})

In [8]:
# 1번에 연관이 있고 2,3,4에는 연관이 없다.
meta.clear()

In [ ]:
Table()
Column()

In [4]:
pk = Column('PK',Integer, primary_key=True)

In [ ]:
Table('User', meta, 
      pk, 
      Column('NAME',Text, nullable=False))

In [9]:
meta.clear()

In [9]:
Table('User', meta, 
      Column('PK',Integer, primary_key=True), 
      Column('NAME',Text, nullable=False))

Table('User', MetaData(), Column('PK', Integer(), table=<User>, primary_key=True, nullable=False), Column('NAME', Text(), table=<User>, nullable=False), schema=None)

In [10]:
meta.tables

FacadeDict({'User': Table('User', MetaData(), Column('PK', Integer(), table=<User>, primary_key=True, nullable=False), Column('NAME', Text(), table=<User>, nullable=False), schema=None)})

In [11]:
meta.create_all(engine)
# 이제 DB가 생성이 된다.

2024-03-08 10:28:49,595 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-08 10:28:49,596 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("User")
2024-03-08 10:28:49,597 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-08 10:28:49,598 INFO sqlalchemy.engine.Engine COMMIT


In [12]:
User = meta.tables['User']

In [13]:
print(select(User)), print(User.select())

SELECT "User"."PK", "User"."NAME" 
FROM "User"
SELECT "User"."PK", "User"."NAME" 
FROM "User"


(None, None)

In [14]:
rst = engine.execute(select(User))

2024-03-08 10:29:03,460 INFO sqlalchemy.engine.Engine SELECT "User"."PK", "User"."NAME" 
FROM "User"
2024-03-08 10:29:03,460 INFO sqlalchemy.engine.Engine [generated in 0.00065s] ()


In [15]:
rst.fetchall()

[]

In [16]:
# 아래 두개의 출력이 다르다. 
print(insert(User))
print(insert(User).values(NAME='어쩌고'))
print(insert(User).values(NAME='어쩌고').compile().params)

INSERT INTO "User" ("PK", "NAME") VALUES (:PK, :NAME)
INSERT INTO "User" ("NAME") VALUES (:NAME)
{'NAME': '어쩌고'}


In [17]:
engine.execute(insert(User).values(NAME='어쩌고'))

2024-03-08 10:29:50,360 INFO sqlalchemy.engine.Engine INSERT INTO "User" ("NAME") VALUES (?)
2024-03-08 10:29:50,360 INFO sqlalchemy.engine.Engine [generated in 0.00041s] ('어쩌고',)
2024-03-08 10:29:50,361 INFO sqlalchemy.engine.Engine COMMIT


In [18]:
for person in ['사람1','사람2','사람3']:
    engine.execute(User.insert().values(NAME=person))

2024-03-08 10:32:02,982 INFO sqlalchemy.engine.Engine INSERT INTO "User" ("NAME") VALUES (?)
2024-03-08 10:32:02,982 INFO sqlalchemy.engine.Engine [cached since 132.6s ago] ('사람1',)
2024-03-08 10:32:02,983 INFO sqlalchemy.engine.Engine COMMIT
2024-03-08 10:32:02,983 INFO sqlalchemy.engine.Engine INSERT INTO "User" ("NAME") VALUES (?)
2024-03-08 10:32:02,984 INFO sqlalchemy.engine.Engine [cached since 132.6s ago] ('사람2',)
2024-03-08 10:32:02,984 INFO sqlalchemy.engine.Engine COMMIT
2024-03-08 10:32:02,985 INFO sqlalchemy.engine.Engine INSERT INTO "User" ("NAME") VALUES (?)
2024-03-08 10:32:02,986 INFO sqlalchemy.engine.Engine [cached since 132.6s ago] ('사람3',)
2024-03-08 10:32:02,986 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
engine.execute(User.select().where(User.c['PK']==1)).fetchall()
engine.execute(User.select().where_or((User.c['PK']==1))).fetchall()

In [21]:
### 다시!

In [22]:
conn = engine.connect()

In [24]:
### 다시

In [25]:
engine = create_engine('sqlite:///playlist.db', echo=True)

In [26]:
conn = engine.connect()

In [27]:
meta = MetaData()

In [28]:
meta.reflect(conn)

2024-03-08 10:36:19,060 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2024-03-08 10:36:19,061 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-08 10:36:19,062 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("ALBUM")
2024-03-08 10:36:19,062 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-08 10:36:19,063 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2024-03-08 10:36:19,063 INFO sqlalchemy.engine.Engine [raw sql] ('ALBUM',)
2024-03-08 10:36:19,064 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("ALBUM")
2024-03-08 10:36:19,064 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-08 10:36:19,065 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("ALBUM")
2024-03-08 10:36:19,065 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-08 10:36:19,065 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sq

In [29]:
len(meta.tables), meta.tables.keys()

(4, dict_keys(['ALBUM', 'ARTIST', 'GENRE', 'TRACK']))

In [30]:
meta.tables['TRACK'].c.keys()

['PK', 'NAME', 'LENGTH', 'RATING', 'COUNT', 'AFK', 'GFK']

In [31]:
Artist = meta.tables['ARTIST']
Album = meta.tables['ALBUM']
Genre = meta.tables['GENRE']
Track = meta.tables['TRACK']

In [33]:
print(Artist.c)

ImmutableColumnCollection(ARTIST.PK, ARTIST.NAME, ARTIST.temp)


In [36]:
print(Artist.select())
conn.execute(Artist.select()).fetchall()

SELECT "ARTIST"."PK", "ARTIST"."NAME", "ARTIST".temp 
FROM "ARTIST"
2024-03-08 10:39:03,149 INFO sqlalchemy.engine.Engine SELECT "ARTIST"."PK", "ARTIST"."NAME", "ARTIST"."temp" 
FROM "ARTIST"
2024-03-08 10:39:03,150 INFO sqlalchemy.engine.Engine [generated in 0.00056s] ()


[(1, '수지', ''), (2, '박효신', ''), (3, '아이유', '')]

In [37]:
print(Album.join(Artist, Album.c.FK==Artist.c.PK))

"ALBUM" JOIN "ARTIST" ON "ALBUM"."FK" = "ARTIST"."PK"


In [40]:
conn.execute(select([Artist.c.NAME], Album.c.NAME).select_from(
    Album.join(Artist, Album.c.FK==Artist.c.PK)
)).fetchall()

2024-03-08 10:42:03,884 INFO sqlalchemy.engine.Engine SELECT "ARTIST"."NAME" 
FROM "ALBUM" JOIN "ARTIST" ON "ALBUM"."FK" = "ARTIST"."PK" 
WHERE "ALBUM"."NAME"
2024-03-08 10:42:03,884 INFO sqlalchemy.engine.Engine [generated in 0.00079s] ()


[]

In [41]:
print(Album.join(Artist, Album.c.FK==Artist.c.PK).join(Track, Album.c.PK==Track.c.AFK))

"ALBUM" JOIN "ARTIST" ON "ALBUM"."FK" = "ARTIST"."PK" JOIN "TRACK" ON "ALBUM"."PK" = "TRACK"."AFK"


In [42]:
conn.execute(select([Artist.c.NAME, Album.c.NAME, Track.c.NAME]).select_from(
    Album.join(Artist, Album.c.FK==Artist.c.PK).join(Track, Album.c.PK==Track.c.AFK)
)).fetchall()

2024-03-08 10:43:41,360 INFO sqlalchemy.engine.Engine SELECT "ARTIST"."NAME", "ALBUM"."NAME" AS "NAME_1", "TRACK"."NAME" AS "NAME_2" 
FROM "ALBUM" JOIN "ARTIST" ON "ALBUM"."FK" = "ARTIST"."PK" JOIN "TRACK" ON "ALBUM"."PK" = "TRACK"."AFK"
2024-03-08 10:43:41,361 INFO sqlalchemy.engine.Engine [generated in 0.00071s] ()


[('박효신', '박효신앨범', '박효신노래1'),
 ('박효신', '박효신앨범', '박효신노래2'),
 ('박효신', '박효신앨범', '박효신노래3'),
 ('아이유', '아이유앨범', '아이유노래1'),
 ('아이유', '아이유앨범', '아이유노래2'),
 ('아이유', '아이유앨범', '아이유노래3')]

In [44]:
print( Album.join(Artist, Album.c.FK==Artist.c.PK) \
      .join(Track, Track.c.AFK==Album.c.PK) \
      .join(Genre, Track.c.GFK==Genre.c.PK) \
    )

"ALBUM" JOIN "ARTIST" ON "ALBUM"."FK" = "ARTIST"."PK" JOIN "TRACK" ON "TRACK"."AFK" = "ALBUM"."PK" JOIN "GENRE" ON "TRACK"."GFK" = "GENRE"."PK"


In [45]:
Artist

Table('ARTIST', MetaData(), Column('PK', INTEGER(), table=<ARTIST>, primary_key=True), Column('NAME', TEXT(), table=<ARTIST>, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x1199d4dd0>, for_update=False)), Column('temp', TEXT(), table=<ARTIST>, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x119a52290>, for_update=False)), schema=None)

In [47]:
Table('ARTIST', meta, Column('PK',Integer))

InvalidRequestError: Table 'ARTIST' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [48]:
meta.clear()
conn.close()

In [49]:
meta, conn

(MetaData(), <sqlalchemy.engine.base.Connection at 0x119ae8f10>)

In [50]:
engine = create_engine('sqlite:///:memory:"', echo=True)
conn = engine.connect()

In [51]:
meta

MetaData()

In [52]:
# 메타에만 등록이 된다.
Table('TEMP', meta, Column('PK', Integer))

Table('TEMP', MetaData(), Column('PK', Integer(), table=<TEMP>), schema=None)

In [54]:
Table('TEMP', meta, Column('PK', Integer, primary_key=True))

InvalidRequestError: Table 'TEMP' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [53]:
# 메타에 테이블이 존재하면 그 테이블을 수정한다.
Table('TEMP', meta, Column('PK', Integer, primary_key=True), extend_existing=True)

Table('TEMP', MetaData(), Column('PK', Integer(), table=<TEMP>, primary_key=True, nullable=False), schema=None)

In [55]:
meta.create_all(conn) # 이렇게 해야 테이블이 생성이 된다. 
# lazy connection 이 이건가? 

2024-03-08 10:51:57,675 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("TEMP")
2024-03-08 10:51:57,676 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-08 10:51:57,677 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("TEMP")
2024-03-08 10:51:57,677 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-08 10:51:57,678 INFO sqlalchemy.engine.Engine 
CREATE TABLE "TEMP" (
	"PK" INTEGER NOT NULL, 
	PRIMARY KEY ("PK")
)


2024-03-08 10:51:57,678 INFO sqlalchemy.engine.Engine [no key 0.00023s] ()
2024-03-08 10:51:57,679 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
Table('TEMP', meta, Column('PK', Integer, primary_key=True))
# ORM은 주로 기존에 있는 DB에 매핑할 때 주로 사용한다. 

In [56]:
# mapper를 사용할 것 이다. 클래스는 다음 주 월요일에. 